In [1]:
import pandas as pd
import torch
import math
import numpy as np
# Load the data
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

# Convert to numpy arrays
train_np = train_df.values
test_np = test_df.values

# Create the train and test tensors
trainX = torch.tensor(train_np[:, :-1], dtype=torch.float64)
trainY = torch.tensor(train_np[:, -1:], dtype=torch.float64)
testX = torch.tensor(test_np[:, :-1], dtype=torch.float64)
testY = torch.tensor(test_np[:, -1:], dtype=torch.float64)

trainX, trainY

(tensor([[ 24.,  51.,  44.,  ...,  61.,  27.,   0.],
         [ 22.,  31., 214.,  ...,   9.,   0.,   0.],
         [ 84., 126., 239.,  ...,  74.,  24.,   8.],
         ...,
         [268., 358., 277.,  ..., 107.,  47.,   7.],
         [ 81., 116.,  90.,  ...,  36.,   9.,   0.],
         [118., 156., 197.,  ...,  19.,   0.,   0.]], dtype=torch.float64),
 tensor([[3.],
         [2.],
         [9.],
         ...,
         [5.],
         [2.],
         [5.]], dtype=torch.float64))

In [2]:
#Q1

1044 * 10 * 8


83520

In [3]:
#Q2
trainXConverted = trainX.to(torch.float16).to(torch.float64)

torch.max(torch.abs(trainX - trainXConverted)).item()

0.0

In [4]:
#Q3
torch.cuda.is_available()


False

In [5]:
#Q4
coef = torch.tensor([
        [0.0040],
        [0.0040],
        [0.0040],
        [0.0040],
        [0.0040],
        [0.0040], # POS_50_59_CP
        [0.0300], # POS_60_69_CP
        [0.0300],
        [0.0300],
        [0.0300]
], dtype=trainX.dtype)
coef

predicted_deaths_first_tract = torch.matmul(testX[0], coef).item()

predicted_deaths_first_tract


9.844

In [6]:
#Q5
average_predicted_deaths = torch.matmul(testX, coef).mean().item()
average_predicted_deaths


12.073632183908048

In [7]:
#Q6
def compute_y(x):
    return x**2 - 8*x + 19
    
x = torch.tensor(0.0)
y = compute_y(x)
float(y)


19.0

In [8]:
#Q7
x = torch.tensor(0.0, requires_grad=True)
optimizer = torch.optim.SGD([x], lr=0.01)   


for _ in range(1000):
    y = compute_y(x)      
    y.backward()          
    optimizer.step()
    optimizer.zero_grad()  
x.item()

3.9999940395355225

In [9]:
#Q8
coef = torch.zeros(trainX.shape[1], 1, dtype=torch.float64, requires_grad=True)

predictions = trainX @ coef

mse_loss = torch.nn.MSELoss()
mse_with_zero_coef = mse_loss(predictions, trainY)
mse_with_zero_coef.item() 


197.8007662835249

In [10]:
#Q9
torch.manual_seed(544)

ds = torch.utils.data.TensorDataset(trainX, trainY)
dl = torch.utils.data.DataLoader(ds, batch_size=50, shuffle=True)

optimizer = torch.optim.SGD([coef], lr=0.000002)

for epoch in range(500):
    for xPrime, yPrime in dl:
        optimizer.zero_grad()
        predictions = xPrime @ coef
        loss = mse_loss(predictions, yPrime)
        loss.backward()
        optimizer.step()
        
train_predictions = trainX @ coef
mse_train = mse_loss(train_predictions, trainY).item()

mse_train


26.8113940147193

In [11]:
#Q10
test_predictions = testX @ coef
mse_test = mse_loss(test_predictions, testY).item()

mse_test

29.05854692548551